In [3]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
print('Libraries imported.')

Libraries imported.


In [8]:
canadasource= requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup=BeautifulSoup(canadasource,'lxml')

In [9]:
tableCanada=soup.find('table',class_='wikitable sortable')

In [10]:
tableContent=tableCanada.find_all ('td')
tableContent[0:9]

[<td>M1A</td>, <td>Not assigned</td>, <td>Not assigned
 </td>, <td>M2A</td>, <td>Not assigned</td>, <td>Not assigned
 </td>, <td>M3A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
 </td>]

In [11]:
postalcode=tableContent[0].text
borough=tableContent[1].text
neighborhood=tableContent[2].text
print(postalcode,borough,neighborhood)
print('Length of tableContent list: ',  len(tableContent))

M1A Not assigned Not assigned

Length of tableContent list:  864


In [12]:
column_names = ['PostalCode','Borough', 'Neighborhood'] 

neighborhoods= pd.DataFrame(columns=column_names)
neighborhoods

,PostalCode,Borough,Neighborhood


In [13]:
i=0
while i <= (len(tableContent)-1):
  postalcode=tableContent[i].text
  borough=tableContent[i+1].text
  neighborhood=tableContent[i+2].text.split('\n')[0]
  neighborhoods = neighborhoods.append({'PostalCode':postalcode,
                                          'Borough': borough,
                                          'Neighborhood': neighborhood
                                          }, ignore_index=True)
  i=i+3

In [14]:
neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [15]:
neighborhoods.drop(neighborhoods[neighborhoods['Borough']=="Not assigned"].index,axis=0, inplace=True)
neighborhoods.head(5)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [16]:
neighborhoods.shape

(211, 3)

In [17]:
TorontoDF=neighborhoods.groupby('PostalCode',as_index=False).agg(lambda x:','.join(set(x)))
TorontoDF.shape

(103, 3)

In [18]:
TorontoDF.loc[TorontoDF['Neighborhood']=='Not assigned','Neighborhood']=TorontoDF.loc[TorontoDF['Neighborhood']=='Not assigned','Borough']

In [19]:
TorontoDF.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Morningside,Guildwood,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [20]:
TorontoDF.shape

(103, 3)

In [26]:
TorontoDF.to_csv("TorontoDFpart1.csv")

In [21]:
!wget -q -O 'toronto_data.csv'  http://cocl.us/Geospatial_data
print('Data downloaded!')

torontoLocationData = pd.read_csv("toronto_data.csv")

Data downloaded!


In [22]:
torontoLocationData.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [23]:
torontoLocationData.shape

(103, 3)

In [27]:
TorontoDF = pd.read_csv("TorontoDFpart1.csv")

In [28]:
TorontoDF.head()

,Unnamed: 0,PostalCode,Borough,Neighborhood
0,0,M1B,Scarborough,"Malvern,Rouge"
1,1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,2,M1E,Scarborough,"Morningside,Guildwood,West Hill"
3,3,M1G,Scarborough,Woburn
4,4,M1H,Scarborough,Cedarbrae
